In [ ]:
import requests
import xml.etree.ElementTree as ET
import csv
from bs4 import BeautifulSoup 

sitemap_index_url = "https://thanhnien.vn/sitemap.xml"

response = requests.get(sitemap_index_url)
if response.status_code == 200:
    root = ET.fromstring(response.content)

    ns = {"ns": "http://www.sitemaps.org/schemas/sitemap/0.9"}

    sitemaps = root.findall("ns:sitemap", ns)

    filtered_sitemaps = sitemaps[3:]

    sitemap_urls = [sitemap.find("ns:loc", ns).text for sitemap in filtered_sitemaps]

    csv_filename = "sitemaps_thanhnien.csv"
    with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["Sitemap URL"])  
        for url in sitemap_urls:
            writer.writerow([url])

    print(f"Đã lưu {len(sitemap_urls)} sitemap vào {csv_filename}")
else:
    print("Không thể tải sitemap index")


Đã lưu 987 sitemap vào sitemaps_thanhnien.csv


In [ ]:
import requests
import xml.etree.ElementTree as ET
import csv

sitemap_urls = []
csv_filename = "sitemaps_thanhnien.csv"

with open(csv_filename, mode="r", encoding="utf-8") as file:
    reader = csv.reader(file)
    next(reader)  
    for row in reader:
        sitemap_urls.append(row[0])

def get_article_links(sitemap_url):
    response = requests.get(sitemap_url)
    if response.status_code == 200:
        root = ET.fromstring(response.content)
        ns = {"ns": "http://www.sitemaps.org/schemas/sitemap/0.9"}
        article_links = [url.find("ns:loc", ns).text for url in root.findall("ns:url", ns)]
        return article_links
    return []

articles_filename = "articles_thanhnien.csv"

with open(articles_filename, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Article URL"]) 

with open(articles_filename, mode="a", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    for sitemap_url in sitemap_urls:
        article_links = get_article_links(sitemap_url)
        if article_links:
            writer.writerows([[link] for link in article_links])
        print(f"Đã lấy {len(article_links)} bài báo từ {sitemap_url}")

print(f"Đã lưu danh sách bài báo vào {articles_filename}")


Đã lấy 951 bài báo từ https://thanhnien.vn/sitemaps/sitemaps-2025-2-1-5.xml
Đã lấy 1110 bài báo từ https://thanhnien.vn/sitemaps/sitemaps-2025-2-6-10.xml
Đã lấy 728 bài báo từ https://thanhnien.vn/sitemaps/sitemaps-2025-2-11-15.xml
Đã lấy 1255 bài báo từ https://thanhnien.vn/sitemaps/sitemaps-2025-1-1-5.xml
Đã lấy 1376 bài báo từ https://thanhnien.vn/sitemaps/sitemaps-2025-1-6-10.xml
Đã lấy 1220 bài báo từ https://thanhnien.vn/sitemaps/sitemaps-2025-1-11-15.xml
Đã lấy 1195 bài báo từ https://thanhnien.vn/sitemaps/sitemaps-2025-1-16-20.xml
Đã lấy 1185 bài báo từ https://thanhnien.vn/sitemaps/sitemaps-2025-1-21-25.xml
Đã lấy 1007 bài báo từ https://thanhnien.vn/sitemaps/sitemaps-2025-1-26-31.xml
Đã lấy 1260 bài báo từ https://thanhnien.vn/sitemaps/sitemaps-2024-12-1-5.xml
Đã lấy 1275 bài báo từ https://thanhnien.vn/sitemaps/sitemaps-2024-12-6-10.xml
Đã lấy 1348 bài báo từ https://thanhnien.vn/sitemaps/sitemaps-2024-12-11-15.xml
Đã lấy 1495 bài báo từ https://thanhnien.vn/sitemaps/sitemap

KeyboardInterrupt: 

In [ ]:
import requests
import csv
from bs4 import BeautifulSoup

articles_filename = "articles_thanhnien.csv"
article_urls = []

with open(articles_filename, mode="r", encoding="utf-8") as file:
    reader = csv.reader(file)
    next(reader)  
    for row in reader:
        article_urls.append(row[0])

def get_article_info(url):
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Lỗi khi truy cập {url}")
        return None

    soup = BeautifulSoup(response.text, "lxml")

    title = soup.find("h1", class_="detail-title").text.strip() if soup.find("h1", class_="detail-title") else ""

    abstract_tag = soup.find("h2", class_=["detail-sapo", "has-audio"])
    abstract = abstract_tag.get_text(separator=" ").strip() if abstract_tag else ""


    content = ""
    article_body = soup.find("div", class_="detail-cmain")
    if article_body:
        paragraphs = article_body.find_all("p")
        filtered_paragraphs = [
            p.text.strip() for p in paragraphs 
            if p.text.strip() and not p.has_attr("data-placeholder")  
        ]
        content = " ".join(filtered_paragraphs)

    category = soup.find("a", class_="category-page__name").text.strip() if soup.find("a", class_="category-page__name") else ""

    return [url, title, abstract, content, category]

output_filename = "thanhnien_articles_data.csv"

with open(output_filename, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["URL", "Title", "Abstract", "Content", "Category"])  

with open(output_filename, mode="a", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    for url in article_urls:
        article_info = get_article_info(url)
        if article_info:
            writer.writerow(article_info)
            print(f"Lấy xong: {url}")

print(f"Hoàn tất! Dữ liệu được lưu vào {output_filename}")


Lấy xong: https://thanhnien.vn/dang-sau-ve-dep-sac-sao-va-quyen-ru-cua-miley-cyrus-tai-grammy-2025-185250205160326938.htm
Lấy xong: https://thanhnien.vn/my-chuan-bi-ke-hoach-rut-quan-khoi-syria-185250205215629875.htm
Lấy xong: https://thanhnien.vn/highlight-clb-nam-dinh-2-1-clb-ha-noi-vong-12-v-league-2024-2025-185250205213912098.htm
Lấy xong: https://thanhnien.vn/clb-thanh-hoa-khong-the-gay-bat-ngo-tai-indonesia-dung-buoc-tai-cup-c1-dong-nam-a-18525020521291745.htm
Lấy xong: https://thanhnien.vn/tong-bi-thu-to-lam-dang-huong-tai-nghia-trang-liet-si-quoc-gia-vi-xuyen-185250205204628537.htm
Lấy xong: https://thanhnien.vn/hlv-nam-dinh-the-do-cho-tuan-hai-la-chinh-xac-hlv-ha-noi-trong-tai-nang-tay-185250205210307533.htm
Lấy xong: https://thanhnien.vn/ca-si-vu-cat-tuong-cau-hon-ban-gai-185250205210831014.htm
Lấy xong: https://thanhnien.vn/ha-noi-xay-dap-dang-de-hoi-sinh-song-to-lich-185250205191422405.htm
Lấy xong: https://thanhnien.vn/chot-tien-do-vanh-dai-phia-tay-noi-dai-cao-toc-hoa-lie

KeyboardInterrupt: 

In [2]:
import pandas as pd
df = pd.read_csv("articles_thanhnien.csv")
df.shape

(1191596, 1)